# Enhanced dynamic form

  📊 Database Tables Created

  - clients - Business information
  - forms - Form configurations
  - lead_sessions - Individual form submissions
  - responses - Question/answer pairs
  - lead_outcomes - Conversion tracking
  - session_snapshots - State recovery

In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal, Dict, Any, Optional
import random
import json
from pathlib import Path
from datetime import datetime
import uuid
import sys
import os

# Add app directory to path for imports (notebook-safe)
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
app_path = os.path.join(project_root, 'app')
if app_path not in sys.path:
    sys.path.append(app_path)

print(f"📁 Project root: {project_root}")
print(f"📁 Source path: {app_path}")

try:
    from database import db
    print("✅ Database module imported successfully")
except ImportError as e:
    print(f"❌ Failed to import database module: {e}")
    print("Creating a mock database connection for now...")

    class MockDB:
        def test_connection(self):
            return False
        def get_lead_session(self, session_id):
            return None
        def create_lead_session(self, data):
            return {}
        def update_lead_session(self, session_id, data):
            return {}
        def get_session_responses(self, session_id):
            return []
        def create_response(self, data):
            return {}

    db = MockDB()

# === Step 1: Enhanced State Definition ===

class LeadState(TypedDict):
    # Session tracking
    session_id: str
    step: int
    form_id: str

    # Question management
    all_questions: List[Dict[str, Any]]
    asked_questions: List[int]  # IDs of questions already asked
    current_step_questions: List[Dict[str, Any]]

    # Response tracking
    responses: List[Dict[str, Any]]  # Each response with question_id, answer, timestamp

    # Lead qualification
    score: int
    lead_status: Literal["unknown", "yes", "maybe", "no"]
    min_questions_met: bool
    failed_required: bool

    # Step content (generated by agents)
    step_headline: str
    step_motivation: str
    phrased_questions: List[str]

    # Session metadata
    started_at: str
    last_updated: str
    completed: bool

# === Database Integration Test ===

def test_database_connection():
    """Test Supabase connection"""
    try:
        print("🔌 Testing Supabase connection with re-enabled legacy keys...")
        success = db.test_connection()
        if success:
            print("✅ Database connection successful!")
            return True
        else:
            print("❌ Database connection failed!")
            return False
    except Exception as e:
        print(f"❌ Database connection error: {e}")
        return False

# Test the connection
db_connected = test_database_connection()

if db_connected:
    print("🎯 Ready to run with real database operations!")
else:
    print("🎯 Running in mock mode - database operations will be simulated")

# === Load Questions and Client Data (Enhanced) ===

def load_questions(data_path: str = "../data/input/dogwalk_34397/questions.json"):
    """Load questions from JSON file"""
    questions_file = Path(data_path)
    with open(questions_file, 'r') as f:
        data = json.load(f)
    return data['questions']

def get_client_info(data_path: str = "../data/input/dogwalk_34397/client.json"):
    """Load client information from JSON file"""
    client_file = Path(data_path)
    with open(client_file, 'r') as f:
        data = json.load(f)
    return data['client']

📁 Project root: /Users/julieberlin/Code/github/pub-aie7-dynamic-surveys
📁 Source path: /Users/julieberlin/Code/github/pub-aie7-dynamic-surveys/app
✅ Database module imported successfully
🔌 Testing Supabase connection with re-enabled legacy keys...
✅ Database connection successful!
🎯 Ready to run with real database operations!


In [ ]:
# === Step 2: Agent and Node Functions (Updated with Real LLM) ===

def start_node(state: LeadState) -> LeadState:
    """Initialize a new lead session"""
    state['session_id'] = str(uuid.uuid4())
    state['step'] = 0
    state['form_id'] = "dogwalk_demo_form"  # Use simple string ID for POC
    state['all_questions'] = load_questions()
    state['asked_questions'] = []
    state['current_step_questions'] = []
    state['responses'] = []
    state['score'] = 0
    state['lead_status'] = "unknown"
    state['min_questions_met'] = False
    state['failed_required'] = False
    state['step_headline'] = ""
    state['step_motivation'] = ""
    state['phrased_questions'] = []
    state['started_at'] = datetime.now().isoformat()
    state['last_updated'] = datetime.now().isoformat()
    state['completed'] = False

    print(f"🚀 Starting new lead session: {state['session_id'][:8]}...")
    print(f"📋 Form: {state['form_id']}")
    return state

def question_selection_agent(state: LeadState) -> LeadState:
    """Agent: Select 1-3 logically related questions for next step using real LLM"""
    
    try:
        # Import the real agent
        from agents.question_selection_agent import invoke_question_selection
        
        # Use the real LLM agent for question selection
        selected_question_ids = invoke_question_selection(state)
        
        # Get the actual question objects
        selected_questions = [q for q in state['all_questions'] 
                            if q['id'] in selected_question_ids]
        
        state['current_step_questions'] = selected_questions
        
        print(f"🤖 LLM Agent selected {len(selected_questions)} questions for step {state['step'] + 1}")
        for q in selected_questions:
            print(f"   - Q{q['id']}: {q['question'][:50]}...")
            
    except Exception as e:
        print(f"⚠️ LLM agent failed, using fallback: {e}")
        # Fallback to simple selection logic
        available_questions = [q for q in state['all_questions']
                              if q['id'] not in state['asked_questions']]
        
        if not available_questions:
            state['current_step_questions'] = []
            return state
            
        # Simple fallback selection
        selected_questions = available_questions[:2]  # Take first 2 available
        state['current_step_questions'] = selected_questions
        
        print(f"📋 Fallback selected {len(selected_questions)} questions for step {state['step'] + 1}")
        for q in selected_questions:
            print(f"   - Q{q['id']}: {q['question'][:50]}...")

    return state

def question_phrasing_node(state: LeadState) -> LeadState:
    """Node: Adapt question phrasing using simple LLM call"""

    # Safety check - if no questions to phrase, return early
    if not state['current_step_questions']:
        print("⚠️  No questions to phrase - skipping phrasing")
        state['phrased_questions'] = []
        return state

    try:
        # Import LLM utility
        from llm_utils import rephrase_questions_simple
        
        client_info = get_client_info()
        
        # Use LLM to rephrase questions
        phrased_questions = rephrase_questions_simple(state['current_step_questions'], client_info)
        state['phrased_questions'] = phrased_questions
        
        print(f"🤖 LLM rephrased {len(phrased_questions)} questions")
        
    except Exception as e:
        print(f"⚠️ LLM phrasing failed, using fallback: {e}")
        # Fallback to simple phrasing
        client_info = get_client_info()
        business_name = client_info['information']['name']
        
        phrased_questions = []
        for q in state['current_step_questions']:
            original = q['question']
            # Simple fallback phrasing
            if 'your dog' in original.lower():
                phrased = original.replace('your dog', 'your furry friend')
            else:
                phrased = original
            phrased_questions.append(phrased)
        
        state['phrased_questions'] = phrased_questions
        print(f"✍️  Fallback phrased {len(phrased_questions)} questions")

    return state

In [ ]:
def engagement_agent(state: LeadState) -> LeadState:
    """Agent: Add engaging headlines and motivational content using real LLM"""

    # Safety check - if no questions to engage with, return early
    if not state['current_step_questions']:
        print("⚠️  No questions for engagement - skipping engagement")
        state['step_headline'] = "Thank you for your responses!"
        state['step_motivation'] = "We have all the information we need."
        return state

    try:
        # Import the real agent
        from agents.engagement_agent import invoke_engagement_agent
        
        client_info = get_client_info()
        
        # Use the real LLM agent for engagement
        engagement_content = invoke_engagement_agent(state, client_info)
        
        state['step_headline'] = engagement_content['headline']
        state['step_motivation'] = engagement_content['motivation']
        
        print(f"🤖 LLM Agent generated engaging content for step {state['step'] + 1}")
        print(f"   Headline: {state['step_headline']}")
        
    except Exception as e:
        print(f"⚠️ LLM engagement agent failed, using fallback: {e}")
        # Fallback engagement content
        client_info = get_client_info()
        business_name = client_info['information']['name']
        step_num = state['step'] + 1
        
        headlines = [
            "Let's find the perfect walking plan for your pup! 🐕",
            "Tell us more about your furry family member 🐾",
            "Almost there! Just a few more details... ✨",
            "Perfect! Let's make sure we're the right fit 🎯",
            "Final step - we're excited to meet you both! 🤝"
        ]
        
        motivations = [
            "Your dog's happiness and health are our top priorities.",
            "Every dog is unique - help us create a personalized experience.",
            "We're building something amazing together!",
            "Great answers so far - you're clearly a caring dog parent.",
            "Almost done! We can't wait to start this journey with you."
        ]
        
        if step_num <= len(headlines):
            state['step_headline'] = headlines[step_num - 1]
            state['step_motivation'] = motivations[step_num - 1]
        else:
            state['step_headline'] = f"Step {step_num} - You're doing great! 🌟"
            state['step_motivation'] = "Thanks for sharing these important details with us."
        
        print(f"🎯 Fallback generated content for step {step_num}")
        print(f"   Headline: {state['step_headline']}")

    return state

def present_step_node(state: LeadState) -> LeadState:
    """Node: Present questions to user and collect responses"""

    # Safety check - if no questions to present, skip this step
    if not state['current_step_questions'] or not state['phrased_questions']:
        print("⚠️  No questions to present - skipping presentation")
        return state

    print(f"\n" + "="*60)
    print(f"✨ {state['step_headline']}")
    print(f"💭 {state['step_motivation']}")
    print("="*60)

    # Simulate user interaction (in real app, this would be frontend)
    for i, (phrased_question, q_data) in enumerate(zip(state['phrased_questions'], state['current_step_questions'])):
        print(f"\n{i+1}. {phrased_question}")

        # Mock responses for demo (in real app, user input)
        mock_answers = {
            1: "Somerville, MA",  # Location
            2: "Golden Retriever",  # Breed
            3: "No, she's well-trained",  # Pulling
            4: "3 times per week",  # Frequency
            5: "Yes, up to date",  # Vaccination
            6: "Parks and trails",  # Environments
            7: "Sarah Johnson",  # Name
            8: "617-555-0123",  # Phone
            9: "email",  # Contact preference
            10: "Peace of mind while at work",  # Benefit
            11: "45 minutes",  # Walk length
            12: "Yes, she loves other dogs",  # Group walks
            13: "No training needed",  # Training interest
            14: "She's perfect!",  # Training needs
            15: "She's the sweetest, most loyal companion"  # Love about dog
        }

        answer = mock_answers.get(q_data['id'], f"Mock answer {q_data['id']}")
        print(f"   → {answer}")

        # Store response with both original and phrased question text
        response = {
            'question_id': q_data['id'],
            'original_question': q_data['question'],  # Original from JSON
            'phrased_question': phrased_question,     # After phrasing agent
            'answer': answer,
            'timestamp': datetime.now().isoformat(),
            'step': state['step'] + 1,
            'data_type': q_data.get('data_type', ''),
            'is_required': q_data.get('is_required', False),
            'scoring_rubric': q_data.get('scoring_rubric', '')
        }
        state['responses'].append(response)
        state['asked_questions'].append(q_data['id'])

    state['step'] += 1
    state['last_updated'] = datetime.now().isoformat()

    return state

def real_database_save_node(state: LeadState) -> LeadState:
    """Node: Save session state to Supabase database"""

    try:
        # Update or create lead session
        session_data = {
            'session_id': state['session_id'],
            'form_id': state['form_id'],
            'final_score': state['score'],
            'lead_status': state['lead_status'],
            'completed': state['completed'],
            'step_count': state['step'],
            'updated_at': state['last_updated']
        }

        # Check if session exists
        existing_session = db.get_lead_session(state['session_id'])

        if existing_session:
            # Update existing session
            result = db.update_lead_session(state['session_id'], session_data)
            print(f"💾 Updated lead session in database")
        else:
            # Create new session (add required fields)
            session_data.update({
                'started_at': state['started_at'],
                'completed_at': datetime.now().isoformat() if state['completed'] else None
            })
            result = db.create_lead_session(session_data)
            print(f"💾 Created new lead session in database")

        # Save any new responses to database
        stored_responses = db.get_session_responses(state['session_id'])
        stored_response_ids = {r['question_id'] for r in stored_responses}

        new_responses = [r for r in state['responses']
                        if r['question_id'] not in stored_response_ids]

        for response in new_responses:
            response_data = {
                'session_id': state['session_id'],
                'question_id': response['question_id'],
                'original_question': response['original_question'],
                'phrased_question': response['phrased_question'],
                'answer': response['answer'],
                'step': response['step'],
                'data_type': response.get('data_type', ''),
                'is_required': response.get('is_required', False),
                'scoring_rubric': response.get('scoring_rubric', ''),
                'timestamp': response['timestamp']
            }

            db.create_response(response_data)
            print(f"   📝 Saved response for Q{response['question_id']}")

        print(f"💾 Database save complete (Step {state['step']}, Score: {state['score']})")

    except Exception as e:
        print(f"❌ Database save failed: {e}")
        # Fall back to mock save
        print(f"💾 Using fallback mock save")

    return state

In [ ]:
def lead_scoring_agent(state: LeadState) -> LeadState:
    """Agent: Score lead using real LLM agent with advanced reasoning"""

    try:
        # Import the real agent
        from agents.lead_scoring_agent import invoke_lead_scoring_agent
        
        # Use the real LLM agent for lead scoring
        scoring_result = invoke_lead_scoring_agent(state, state['all_questions'])
        
        # Update state with LLM results
        state['score'] = scoring_result['score']
        state['min_questions_met'] = len(state['responses']) >= 4
        
        # Determine lead status with business rules
        if any("CRITICAL" in flag for flag in scoring_result.get('red_flags', [])):
            state['lead_status'] = "no"
            state['failed_required'] = True
        elif not state['min_questions_met']:
            state['lead_status'] = "unknown"
        elif state['score'] >= 80:
            state['lead_status'] = "yes"
        elif state['score'] <= 25:
            state['lead_status'] = "no"
        else:
            state['lead_status'] = "maybe"
        
        print(f"🤖 LLM Agent Scoring Results:")
        print(f"   Score: {state['score']}/100")
        print(f"   Status: {state['lead_status'].upper()}")
        print(f"   Reasoning: {scoring_result.get('reasoning', 'No reasoning provided')}")
        if scoring_result.get('positive_indicators'):
            print(f"   ✅ Positives: {', '.join(scoring_result['positive_indicators'])}")
        if scoring_result.get('red_flags'):
            print(f"   ⚠️ Concerns: {', '.join(scoring_result['red_flags'])}")
        
    except Exception as e:
        print(f"⚠️ LLM scoring agent failed, using fallback: {e}")
        # Fallback to original scoring logic
        total_score = 0
        scoring_details = []

        # Check minimum questions requirement
        state['min_questions_met'] = len(state['responses']) >= 4

        for response in state['responses']:
            question_id = response['question_id']
            answer = response['answer'].lower()

            # Find the question data for scoring rubric
            question_data = next((q for q in state['all_questions'] if q['id'] == question_id), None)
            if not question_data or not question_data.get('scoring_rubric'):
                continue

            rubric = question_data['scoring_rubric'].lower()
            points = 0

            # Apply scoring rubric logic
            if question_id == 1:  # Location
                if 'somerville' in answer or 'cambridge' in answer or 'boston' in answer:
                    points = 20  # Within 20 minutes
                elif 'ma' in answer or 'massachusetts' in answer:
                    points = 10  # Reasonable distance
                else:
                    points = -10  # Too far

            elif question_id == 2:  # Dog breed
                if 'german shepherd' in answer:
                    points = 25  # Highest score
                elif any(dangerous in answer for dangerous in ['pit bull', 'rottweiler', 'doberman']):
                    points = -20  # Dangerous breeds
                else:
                    points = 15  # Normal breeds

            elif question_id == 3:  # Pulling behavior
                if 'no' in answer or 'well-trained' in answer:
                    points = 15  # Good behavior
                else:
                    points = -5  # Pulls when walking

            elif question_id == 4:  # Frequency
                if '5 times' in answer:
                    points = 10
                elif '3' in answer or '4' in answer:
                    points = 20  # Ideal frequency
                elif '2' in answer:
                    points = 15
                elif 'once' in answer:
                    points = 5  # Lowest score but acceptable

            elif question_id == 5:  # Vaccination
                if 'yes' in answer or 'up to date' in answer:
                    points = 25  # Must be vaccinated
                else:
                    points = -50  # Hard fail
                    state['failed_required'] = True

            elif question_id == 12:  # Group walks
                if 'yes' in answer:
                    points = 15  # Positive for group walks
                else:
                    points = 5

            total_score += points
            scoring_details.append(f"Q{question_id}: {points} points")

        state['score'] = max(0, total_score)  # Don't go below 0

        # Determine lead status based on business rules
        if state['failed_required']:
            state['lead_status'] = "no"
        elif not state['min_questions_met']:
            state['lead_status'] = "unknown"  # Need more questions
        elif state['score'] >= 80:
            state['lead_status'] = "yes"
        elif state['score'] <= 25:
            state['lead_status'] = "no"
        else:
            state['lead_status'] = "maybe"

        print(f"📊 Fallback Lead Scoring Results:")
        print(f"   Score: {state['score']}/100")
        print(f"   Status: {state['lead_status'].upper()}")
        print(f"   Questions answered: {len(state['responses'])}")
        print(f"   Minimum met: {state['min_questions_met']}")
        if state['failed_required']:
            print(f"   ⚠️  Failed required question")

    return state

def completion_node(state: LeadState) -> LeadState:
    """Generate personalized completion message and finalize lead"""

    state['completed'] = True
    state['last_updated'] = datetime.now().isoformat()

    client_info = get_client_info()
    business_name = client_info['information']['name']
    owner_name = client_info['information']['owner']

    # Generate personalized messages based on lead status and responses
    dog_name = "your pup"
    owner_name_response = next((r['answer'] for r in state['responses']
                               if r['question_id'] == 7), "")
    if owner_name_response:
        greeting = f"Hi {owner_name_response.split()[0]}!"
    else:
        greeting = "Hello!"

    # Find dog breed for personalization
    breed_response = next((r['answer'] for r in state['responses']
                          if r['question_id'] == 2), "")
    if breed_response:
        dog_name = f"your {breed_response}"

    if state['lead_status'] == "yes":
        message = f"""
🎉 {greeting} Wonderful news!

Based on your responses, {business_name} would be thrilled to provide walking services for {dog_name}!

{owner_name} will personally reach out within 24 hours to discuss scheduling and answer any questions you might have.

We're excited to meet you both and provide the best care for your furry family member! 🐾
        """

    elif state['lead_status'] == "maybe":
        message = f"""
🤔 {greeting} Thank you for your interest!

We'd love to learn a bit more about {dog_name} to ensure we're the perfect fit.

{owner_name} will review your responses and get back to you within 48 hours with next steps.

We appreciate you considering {business_name} for your dog walking needs! 🐕
        """

    else:  # "no" status or "unknown"
        message = f"""
🙏 {greeting} Thank you for your time!

While we may not be the perfect fit for {dog_name} at this time, we appreciate you considering {business_name}.

We'd be happy to provide recommendations for other services that might better meet your needs.

Best wishes to you and your furry friend! 🐾
        """

    print(f"\n" + "="*70)
    print("🏁 FORM COMPLETION")
    print("="*70)
    print(message)
    print(f"\n📈 Final Score: {state['score']}/100")
    print(f"🎯 Lead Status: {state['lead_status'].upper()}")
    print(f"📊 Questions Answered: {len(state['responses'])}")

    # Mock real-time notification for "yes" leads
    if state['lead_status'] == "yes":
        print(f"\n🔔 REAL-TIME ALERT: New qualified lead! Session {state['session_id'][:8]}")
    elif state['lead_status'] == "maybe":
        print(f"\n📝 BATCH QUEUE: Added to daily review list")

    return state

In [14]:
# === Step 3: Build Enhanced LangGraph with Real Database ===

builder = StateGraph(LeadState)

# Add all nodes
builder.add_node("start", start_node)
builder.add_node("question_selection", question_selection_agent)
builder.add_node("question_phrasing", question_phrasing_node)
builder.add_node("engagement", engagement_agent)
builder.add_node("present_step", present_step_node)
builder.add_node("database_save", real_database_save_node)  # Real database operations
builder.add_node("lead_scoring", lead_scoring_agent)
builder.add_node("completion", completion_node)

# Set entry point
builder.set_entry_point("start")

# Define the main flow
builder.add_edge("start", "question_selection")

# Add conditional routing IMMEDIATELY after question selection to check if we have questions
def check_questions_available(state: LeadState) -> str:
    """Check if we have questions before proceeding"""
    if not state['current_step_questions']:
        print("🚫 No questions selected - routing to completion")
        return "completion"
    print("✅ Questions available - continuing flow")
    return "continue_flow"

builder.add_conditional_edges(
    "question_selection",
    check_questions_available,
    {
        "completion": "completion",
        "continue_flow": "question_phrasing"
    }
)

builder.add_edge("question_phrasing", "engagement")
builder.add_edge("engagement", "present_step")
builder.add_edge("present_step", "database_save")  # Use real database save
builder.add_edge("database_save", "lead_scoring")

# Add conditional routing after scoring with more restrictive logic
def routing_decision_fixed(state: LeadState) -> str:
    """Determine next step in the flow with better safety checks"""

    print(f"\n🔍 ROUTING DECISION DEBUG:")
    print(f"   Questions answered: {len(state['responses'])}")
    print(f"   Lead status: {state['lead_status']}")
    print(f"   Failed required: {state['failed_required']}")
    print(f"   Min questions met: {state['min_questions_met']}")

    # First check if we have more questions available
    available_questions = [q for q in state['all_questions']
                          if q['id'] not in state['asked_questions']]
    print(f"   Available questions: {len(available_questions)}")

    # FORCE completion in these cases
    if not available_questions:
        print("🔚 DECISION: No questions available - COMPLETING")
        return "completion"

    if state['failed_required']:
        print("❌ DECISION: Failed required question - COMPLETING")
        return "completion"

    if len(state['responses']) >= 6:  # Lower safety limit
        print("🛑 DECISION: Safety limit reached - COMPLETING")
        return "completion"

    if state['lead_status'] == "yes" and len(state['responses']) >= 4:
        print("✅ DECISION: High-quality lead with minimum questions - COMPLETING")
        return "completion"

    if state['lead_status'] == "no" and len(state['responses']) >= 4:
        print("❌ DECISION: Clear no-lead with minimum questions - COMPLETING")
        return "completion"

    # Only continue if we really need more info AND have questions available
    if len(state['responses']) < 4 or state['lead_status'] == "unknown":
        print("➡️ DECISION: Need more information - CONTINUING")
        return "continue"

    # Default to completion as safety
    print("🔚 DECISION: Default safety - COMPLETING")
    return "completion"

builder.add_conditional_edges(
    "lead_scoring",
    routing_decision_fixed,
    {
        "continue": "question_selection",
        "completion": "completion"
    }
)

# Completion is the end
builder.add_edge("completion", END)

# Compile the graph
app = builder.compile()

print("✅ Enhanced LangGraph with Real Database compiled successfully!")
print("\nFlow: Start → Question Selection → [Check Available] → Phrasing → Engagement → Present → **Database Save** → Score → [Routing Decision]")

✅ Enhanced LangGraph with Real Database compiled successfully!

Flow: Start → Question Selection → [Check Available] → Phrasing → Engagement → Present → **Database Save** → Score → [Routing Decision]


In [15]:
# Display the graph structure
app

In [17]:
# === Step 4: Test the Enhanced Flow ===

def run_demo_scenario(scenario_name: str):
    """Run a demo scenario with the enhanced flow"""

    print(f"\n{'='*80}")
    print(f"🎬 DEMO SCENARIO: {scenario_name}")
    print(f"{'='*80}")

    # Initialize state (this would normally be empty)
    initial_state: LeadState = {
        "session_id": "",
        "step": 0,
        "form_id": "",
        "all_questions": [],
        "asked_questions": [],
        "current_step_questions": [],
        "responses": [],
        "score": 0,
        "lead_status": "unknown",
        "min_questions_met": False,
        "failed_required": False,
        "step_headline": "",
        "step_motivation": "",
        "phrased_questions": [],
        "started_at": "",
        "last_updated": "",
        "completed": False
    }

    # Run the flow
    final_state = app.invoke(initial_state)

    print(f"\n🎯 SCENARIO COMPLETE: {scenario_name}")
    print(f"Final Status: {final_state['lead_status'].upper()}")
    print(f"Session ID: {final_state['session_id'][:8]}...")

    return final_state

# Run the demo
if __name__ == "__main__":
    result = run_demo_scenario("High-Quality Dog Owner (Golden Retriever)")

    print(f"\n📊 FINAL SUMMARY:")
    print(f"   • Total Questions Asked: {len(result['responses'])}")
    print(f"   • Final Score: {result['score']}/100")
    print(f"   • Lead Classification: {result['lead_status'].upper()}")
    print(f"   • Session Completed: {result['completed']}")


🎬 DEMO SCENARIO: High-Quality Dog Owner (Golden Retriever)
🚀 Starting new lead session: 36af44ac...
📋 Form ID: 2b5ca753...
📋 Selected 2 questions for step 1
   - Q3: Does your dog pull when walking on a leash?...
   - Q4: How many times per week do you want your dog walke...
✅ Questions available - continuing flow
✍️  Phrased 2 questions for friendly and professional tone
🎯 Generated engaging content for step 1
   Headline: Let's find the perfect walking plan for your pup! 🐕

✨ Let's find the perfect walking plan for your pup! 🐕
💭 Your dog's happiness and health are our top priorities.

1. We'd love to know: Does your furry friend pull when walking on a leash?
   → No, she's well-trained

2. How many times per week do you want your furry friend walked?
   → 3 times per week
💾 Created new lead session in database
   📝 Saved response for Q3
   📝 Saved response for Q4
💾 Database save complete (Step 1, Score: 0)
📊 Lead Scoring Results:
   Score: 35/100
   Status: UNKNOWN
   Questions answ

In [9]:
# === Additional Test Scenarios ===

def test_different_scenarios():
    """Test various lead scenarios to validate business rules"""

    # Override mock answers for different scenarios
    scenarios = [
        {
            "name": "Perfect Lead - German Shepherd",
            "answers": {
                1: "Somerville, MA",
                2: "German Shepherd",
                3: "No, very well trained",
                4: "3 times per week",
                5: "Yes, all vaccinations current"
            }
        },
        {
            "name": "Failed Vaccination Requirement",
            "answers": {
                1: "Cambridge, MA",
                2: "Labrador",
                3: "No pulling issues",
                5: "No, not vaccinated"  # Hard fail
            }
        },
        {
            "name": "Maybe Lead - Needs Review",
            "answers": {
                1: "Worcester, MA",  # Farther away
                2: "Mixed breed",
                3: "Yes, pulls sometimes",
                4: "2 times per week",
                5: "Yes, vaccinated"
            }
        }
    ]

    print("\n" + "🧪 RUNNING MULTIPLE TEST SCENARIOS" + "\n" + "="*50)

    for scenario in scenarios:
        print(f"\n🎭 Testing: {scenario['name']}")
        print("="*40)

        # This would require modifying the mock answers in present_step_node
        # For now, just showing the concept
        print(f"Mock answers: {scenario['answers']}")
        print("✅ Scenario setup ready")

# Display scenario testing info
test_different_scenarios()

print("""
🎯 KEY BUSINESS RULES IMPLEMENTED:

✅ Question Selection Agent:
   • Avoids repeating questions
   • Groups related questions (contact info, dog details, service preferences)
   • Delays tough qualifying questions until after 4 responses
   • Selects 1-3 questions per step maximum

✅ Lead Scoring Agent:
   • Enforces 4-question minimum before lead can fail
   • Uses JSON scoring rubrics for each question
   • Handles required question failures (vaccination)
   • Categorizes leads: yes (80+), maybe (26-79), no (≤25)

✅ Engagement Agent:
   • Creates compelling headlines for each step
   • Adds motivational content to prevent abandonment
   • Adapts messaging based on progress

✅ Auto-save & State Management:
   • Saves state after each step
   • Tracks individual responses for analytics
   • Maintains session continuity
   • Records timestamps and metadata

✅ Personalized Completion:
   • Custom messages based on lead status
   • Uses collected data (name, dog breed) for personalization
   • Different notification strategies (real-time vs batch)
   • Business-appropriate tone and next steps
""")


🧪 RUNNING MULTIPLE TEST SCENARIOS

🎭 Testing: Perfect Lead - German Shepherd
Mock answers: {1: 'Somerville, MA', 2: 'German Shepherd', 3: 'No, very well trained', 4: '3 times per week', 5: 'Yes, all vaccinations current'}
✅ Scenario setup ready

🎭 Testing: Failed Vaccination Requirement
Mock answers: {1: 'Cambridge, MA', 2: 'Labrador', 3: 'No pulling issues', 5: 'No, not vaccinated'}
✅ Scenario setup ready

🎭 Testing: Maybe Lead - Needs Review
Mock answers: {1: 'Worcester, MA', 2: 'Mixed breed', 3: 'Yes, pulls sometimes', 4: '2 times per week', 5: 'Yes, vaccinated'}
✅ Scenario setup ready

🎯 KEY BUSINESS RULES IMPLEMENTED:

✅ Question Selection Agent:
   • Avoids repeating questions
   • Groups related questions (contact info, dog details, service preferences)
   • Delays tough qualifying questions until after 4 responses
   • Selects 1-3 questions per step maximum

✅ Lead Scoring Agent:
   • Enforces 4-question minimum before lead can fail
   • Uses JSON scoring rubrics for each ques